In [1]:
import numpy as np
import pandas as pd

#remove the row and column label from excel sheet before loading

#dataset import for regressor
X_dem=pd.read_excel(r'C:\Users\fhasan1\dataset39.xlsx', header=None, sheet_name=0)
y_gen=pd.read_excel(r'C:\Users\fhasan1\dataset39.xlsx', header=None, sheet_name=1)

#Changing pandas dataframe to numpy array
X_dem = X_dem.to_numpy()
y_gen = y_gen.to_numpy()

# load new voltage and brnach status dataset for classification
y_bus=pd.read_excel(r'C:\Users\fhasan1\dataset39.xlsx', header=None, sheet_name=2)#outputbus
y_branch=pd.read_excel(r'C:\Users\fhasan1\dataset39.xlsx', header=None, sheet_name=3)#outputbranch

#Changing pandas dataframe to numpy array
y_bus = y_bus.to_numpy()
y_branch = y_branch.to_numpy()

# Generator bus indices
Gen_index=pd.read_excel(r'C:\Users\fhasan1\dataset39.xlsx', header=None, sheet_name=4)
Gen_index=np.array(Gen_index)

# Loading Index_of_nonzero_nodal_demand
load_index=pd.read_excel(r'C:\Users\fhasan1\dataset39.xlsx', header=None, sheet_name=5)
load_index=np.array(load_index)
load_index=load_index-1 #as pyhton starts counting from zero
load_index=load_index.T

#number bus branch and gen
nb=np.size(y_bus,1)#bus
nl=np.size(y_branch,1)#branch
ng=len(Gen_index)#gen

print(nb)
print(nl)
print(ng)

39
46
10


In [2]:
# load and evaluate a saved model
from keras.models import load_model
 
# load model
modelG = load_model('modelG_39.h5')
modelV = load_model('modelV_39.h5')
modelB = load_model('modelB_39.h5')

Using TensorFlow backend.


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [4]:
#dataset for classifier trainig
X_classifier=X_dem[2000:4000,0:]
y_classifier_bus=y_bus[2000:4000,0:]
y_classifier_branch=y_branch[2000:4000,0:]
print(X_classifier.shape)
print(y_classifier_bus.shape)
print(y_classifier_branch.shape)

(2000, 78)
(2000, 39)
(2000, 46)


In [5]:
# Generation prediction
X_classifier_dem=np.copy(X_classifier)
X_classifier_dem=X_classifier_dem[:,load_index[0,0:]]  

print(X_classifier_dem.shape)
print(X_classifier.shape)

(2000, 42)
(2000, 78)


In [6]:
#Train test split of model
from sklearn.model_selection import train_test_split
# model for voltage
X_trainV,X_testV,y_trainV,y_testV = train_test_split(X_classifier_dem,y_classifier_bus,test_size = 0.1,random_state = 0)
# model for branch
X_trainB,X_testB,y_trainB,y_testB = train_test_split(X_classifier_dem,y_classifier_branch,test_size = 0.1,random_state = 0)

In [7]:
#bus slassifier model
modelDV = Sequential()
modelDV.add(Dense(256, input_dim=np.size(X_classifier_dem,1), activation='sigmoid'))
modelDV.add(Dense(nb, activation='softmax'))
modelDV.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#branch slassifier model
modelDB = Sequential()
modelDB.add(Dense(256, input_dim=np.size(X_classifier_dem,1), activation='sigmoid'))
modelDB.add(Dense(nl, activation='softmax'))
modelDB.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
historyDV = modelDV.fit(X_trainV, y_trainV, epochs=1000, batch_size=100,verbose=0,validation_split=0.2)
historyDB = modelDB.fit(X_trainB, y_trainB, epochs=1000, batch_size=100,verbose=0,validation_split=0.2)

In [9]:
y_predDV=modelDV.predict(X_testV)
y_predDB=modelDB.predict(X_testB)
print(X_testB.shape)
print(X_testV.shape)

(200, 42)
(200, 42)


In [10]:
#Final testing
#dataset for final testing
X_test_dem=X_dem[4000:,0:]
y_test_gen =y_gen[4000:,0:]
y_test_bus=y_bus[4000:,0:]
y_test_branch=y_branch[4000:,0:]
print(X_test_dem.shape)
print(y_test_gen.shape)
print(y_test_bus.shape)
print(y_test_branch.shape)

(882, 78)
(882, 20)
(882, 39)
(882, 46)


In [11]:
#Filtering zeros from demand
X_test_dem_temp=X_test_dem[:,load_index[0,0:]]

print(X_test_dem.shape)
print(X_test_dem_temp.shape)

# Generation prediction 
y_predG=modelG.predict(X_test_dem_temp)

#converting (ng*1) to (nb*1) matrix
#As pyhton indexing start from zero
Gen_index=Gen_index-1
y_predG_temp=np.zeros([np.size(X_test_dem,0),nb*2], dtype = int)
for s in range(np.size(X_test_dem,0)):
    for i in range(ng):    
        y_predG_temp[s,Gen_index[i]]=y_predG[s,i]
        y_predG_temp[s,Gen_index[i]+nb]=y_predG[s,i+ng] 
           
#forming net injection vectors
X_NI_test=y_predG_temp-X_test_dem

# bus and branch prediction 
y_predV=modelV.predict(X_NI_test)
y_predB=modelB.predict(X_NI_test)

y_predDV=modelDV.predict(X_test_dem_temp)
y_predDB=modelDB.predict(X_test_dem_temp)

(882, 78)
(882, 42)


In [12]:
# testing the trained regressor

#testing the trained voltage classifier
from sklearn.metrics import confusion_matrix

y_testV=np.reshape(y_test_bus,np.size(X_test_dem,0)*nb)
y_predV=np.reshape(y_predV,np.size(X_test_dem,0)*nb)

y_predV[y_predV >= 0.001] = 1
y_predV[y_predV < 0.001] = 0

print(confusion_matrix(y_testV, y_predV))

#testing the trained branch classifier
y_testB=np.reshape(y_test_branch,np.size(X_test_dem,0)*nl)
y_predB=np.reshape(y_predB,np.size(X_test_dem,0)*nl)

y_predB[y_predB >= 0.001] = 1
y_predB[y_predB < 0.001] = 0

print(confusion_matrix(y_testB, y_predB))

[[24086  5272]
 [   14  5026]]
[[38807    25]
 [    1  1739]]


In [13]:
# testing the DB trained regressor
y_predDV=np.reshape(y_predDV,np.size(X_test_dem,0)*nb)

y_predDV[y_predDV >= 0.001] = 1
y_predDV[y_predDV < 0.001] = 0

print(confusion_matrix(y_testV, y_predDV))

#testing the trained branch classifier
y_predDB=np.reshape(y_predDB,np.size(X_test_dem,0)*nl)

y_predDB[y_predDB >= 0.001] = 1
y_predDB[y_predDB < 0.001] = 0

print(confusion_matrix(y_testB, y_predDB))

[[24869  4489]
 [   16  5024]]
[[38807    25]
 [    1  1739]]
